In [1]:
import json
with open('configuration/aml_workspace.json', 'r') as f:
    aml_config = json.load(f)

with open('configuration/train_config.json', 'r') as f:
    train_config = json.load(f)

subscription_id = aml_config['subscription_id']
resource_group = aml_config['resource_group']
workspace_name = aml_config['workspace_name']
datastore_name = train_config['datastore_name']
train_environment = train_config['train_environment']
path_on_datastore = train_config['path_on_datastore']
computename = aml_config['computename']

In [2]:
from azure.ai.ml import MLClient
from azure.ai.ml._restclient.dataset_dataplane.models import Dataset
from azure.identity import (
    AzureCliCredential,
    InteractiveBrowserCredential
)
try:
    # Use AzureCLICredential instead of DefaultAzureCredential since VisualStudioCodeCredential might cause errors.
    # Requires previously logging in to Azure via "az login", and will use the CLI's currently logged in identity.
    credential = AzureCliCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception:
    # Fall back to InteractiveBrowserCredential in case AzureCliCredential not work
    credential = InteractiveBrowserCredential()

# Note that MLCLient is lazy and does not yet connect to Azure here.
# In case the CLI authentication failed, a browser window will pop up later and will require authentication.
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

from azure.ai.ml import command
from azure.ai.ml.entities import Data, Workspace, Environment, BuildContext
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [4]:
my_job_inputs = {
    "input_data": Input(type=AssetTypes.URI_FOLDER, path=f"azureml://datastores/{datastore_name}/paths/{path_on_datastore}")
}
my_job_outputs = {
    "yolo_data": Output(type=AssetTypes.URI_FOLDER, path=f"azureml://datastores/{datastore_name}/paths/output/")
}
job = command(
    code="./src",
    command="python prepare_data_files.py --input_data ${{inputs.input_data}}",
    inputs=my_job_inputs,
    environment=train_environment,
    compute='Standard-L8s-v2',
)
returned_job = ml_client.create_or_update(job)
print(returned_job.services["Studio"].endpoint)

https://ml.azure.com/runs/mango_door_th61qzfh9r?wsid=/subscriptions/83febb18-9ee8-4c49-859f-9e7a106512a5/resourcegroups/dev-000-euw-sordiai-rg/workspaces/de-sordiai-00-ml&tid=659c3a90-ec90-4772-b772-cebe2050a82e
